# Importing some packages

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.utils.data.dataset import random_split
from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

# Model training

Creating higher-order function for a training step

In [5]:
def make_train_step_fn(model, optimizer, lossfn):

    def perform_train_step(x, y):

        model.train()

        y_hat = model(x)

        loss = lossfn(y_hat, y)

        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        return loss.item()
    
    return perform_train_step

Running data preparation

In [13]:
N = 100
ERROR_SCALE = 0.1
TRUE_B = 1
TRUE_W = 2

np.random.seed(42)

X = np.random.rand(N, 1)
error = ERROR_SCALE * np.random.randn(N, 1)
y = TRUE_B + TRUE_W * X + error

idx = np.arange(N)
np.random.shuffle(idx)

train_idx = idx[:int(N*.8)]
val_idx = idx[int(N*.8):]

X_train, y_train = X[train_idx], y[train_idx]
X_val, y_val = X[val_idx], y[val_idx]

device = 'cuda' if torch.cuda.is_available() else 'cpu'

X_train_tensor = torch.as_tensor(X_train).float().to(device)
X_val_tensor = torch.as_tensor(X_val).float().to(device)

y_train_tensor = torch.as_tensor(y_train).float().to(device)
y_val_tensor = torch.as_tensor(y_val).float().to(device)

In [7]:
%run -i data_preparation/v0.py

Creating the model configuration

In [17]:
%%writefile model_configuration/v1.py

lr = 0.1

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1, 1)).to(device)

optimizer = optim.SGD(model.parameters(), lr=lr)

lossfn = nn.MSELoss(reduction="mean")

train_step_fn = make_train_step_fn(model, optimizer, lossfn)

Overwriting model_configuration/v1.py


In [18]:
%run -i model_configuration/v1.py

In [19]:
train_step_fn

<function __main__.make_train_step_fn.<locals>.perform_train_step(x, y)>

Creating model training

In [20]:
%%writefile model_training/v1.py

epochs = 1000

losses = []

for _ in range(epochs):

    loss = train_step_fn(X_train_tensor, y_train_tensor)

    losses.append(loss)

Overwriting model_training/v1.py


In [21]:
%run -i model_training/v1.py

In [22]:
model.state_dict()

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')),
             ('0.bias', tensor([1.0235], device='cuda:0'))])